# Rede Neural Artificial - Classificação Binária

In [2]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers

In [3]:
tf.__version__

'2.0.0-beta1'

# Importar Dados

In [2]:
X = pd.read_csv("entradas-breast.csv")

In [3]:
X.head(2)

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,186.0000,275.0000,0.08902


In [4]:
y = pd.read_csv("saidas-breast.csv")

In [5]:
y.head(2)

,0
0,0
1,0


## Divisão Treino-Teste

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [8]:
len(X_train)

426

In [9]:
len(X_test)

143

## Modelo Artificial Neural Network (Modelo de 1 camada)

In [10]:
#https://stats.stackexchange.com/questions/207049/neural-network-for-binary-classification-use-1-or-2-output-neurons

# feed forward - Numa rede neural feed forward a informação move-se sempre num sentido, nunca volta atrás.(desde a camada de 
# entrada, para as camadas ocultas, se existirem, até a camada de saida) É uma rede neural onde as conecções entre os nodos não
# formam um ciclos, ao contrario das redes neurais recorrentes.

# Back-Propagation - Algoritmo utilizado no treino(supervisionado) da rede feedforward. São atribuidos pesos aos neuronios da 
# rede e os calculos são efectuados com esses pesos partindo da camada de entrada até a camada de saida. 
# Calcula-se o erro obtido na camada de saida e modificam-se os pesos da iteração anterior partindo da camada de saida até a 
# camada de entrada. Utiliza-se o método da descida do gradiente para encontrar o melhor valor para os pesos.
# De seguida, e com os pesos obtidos na iteração anterior, são efectuados os mesmos calculos. Cada actualização dos pesos
# chama-se de epoca.

# Sequential - A informação move-se da camada de entrada, para a camada oculta e de seguida para a camada de saida.

#Dense - Camadas densas(cada um dos neurónios é ligado com todos os neuronios da camada seguinte) tambem chamada de rede neural
# fully connected

# 1ª Camada Oculta(e definição da camada de entrada) 
# Neuronios: normalmente escolhe-se o número de neuronios igual ao número de variaveis de entrada mais o número de variaveis de saida 
# divididos por 2 ((30+1)/2=16) para começar a modelar a rede. Posteriormente pode-se ajustar para se obter melhores resultados
# Função de Activação: Normalmente escolhe-se a reLu para deep learning porque obtem-se melhores resultados do que com uma
# função sigmoide ou tangente hiperbolica. 
# Kernel initializer: Como se inicializam os pesos.
# input_dim: quantos atributos existem na camada de entrada (neste caso são 30 variaveis de entrada). Este parâmetro só é 
# necessário para a primeira camada oculta. Com este parâmetro activo não é necessário colocar explicitamente a camada de 
# entrada porque na 1ª camada oculta está-se a dizer o numero de neurónios da camada de entrada.

# Camada de Saida
# Neuronios: Igual ao número de saidas possiveis, para classificação binária normalmente utiliza-se 1. Considera-se o resultado 
# obtido ser da classe A se for 0 (<0.5) e 1 (>=0.5) é considerado classe B (no caso de se usar a função sigmoid nesta camada).
# Função de activação: Sigmoid porque é um problema de classificação binário que varia entre 0 e 1, e esta função retorna sempre
# um valor entre 0 e 1.

model = models.Sequential()
model.add(layers.Dense(units=16, activation="relu", kernel_initializer="random_uniform", input_dim=30))
model.add(layers.Dense(units=1, activation="sigmoid"))


### Compilação e Ajuste do modelo ANN

In [11]:
#https://medium.com/ensina-ai/uma-explica%C3%A7%C3%A3o-visual-para-fun%C3%A7%C3%A3o-de-custo-binary-cross-entropy-ou-log-loss-eaee662c396c

# Optimizer: adam é uma optimização da descida do gradiente estócastico (usado para encontrar os melhores valores dos pesos).
# É o que melhor se adapta à maioria dos casos.
# loss = função de perda binary_crossentropy é a mais utilizada para classificação binária. Mede o quão afastada está a previsão
# do seu valor real(0 ou 1) para cada classe e faz a média desses erros(desvios) para obter o custo(loss). Como utiliza o
# logaritmo tem uma penalização maior quando existe uma classificação errada.

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["binary_accuracy"])

In [15]:
# batch_size
# Se escolher-se 1(Stochastic gradient descent) calcula-se o erro para cada dado e atualiza-se os pesos de seguida.
# Se escolher-se igual ao numero de dados (batch gradient descent) calcula-se o erro para todos os registros e só depois
# é que se actualizam os pesos.
# Se escolher-se um número intermédio(Mini batch gradient descent), por ex. 10, calcula-se o erro para um conjunto de 10 dados 
# e actualizam-se os pesos (Stochastic gradient descent).

# epochs:  Quantas vezes o algoritmo percorre o conjunto de dados de treino totalmente.
# Caso a base de dados de treino tenha 100 dados.
# Stochastic gradient descent:
# 1ª época: Calcula o erro para o 1º registro e actualiza todos os pesos. Calcula o erro para o 2º registro e actualiza todos 
# os pesos.  Calcula o erro para o 3º registro e actualiza todos os pesos.  E assim sucessivamente até aos 100 registros.
# 2ª época: Calcula o erro para o 1º registro e actualiza todos os pesos da 1ª época. Calcula o erro para o 2º registro e 
# actualiza todos os pesos da iteração anterior.  Calcula o erro para o 3º registro e actualiza todos os pesos da iteração 
# anterior.  E assim sucessivamente.
# batch gradient descent:
# 1ª época: Calcula o erro para o 1º registo. Calcula o erro para o seguindo registo. E assim sucessivamente até ter calculado
# o erro para os 100 registro. No fim actualiza todos os pesos.
# 2ª época: Calcula o erro para o 1º registo. Calcula o erro para o seguindo registo. E assim sucessivamente até ter calculado
# o erro para todos os registro. No fim actualiza todos os pesos da 1ª época.
# Mini batch gradient descent (por ex. batch=10):
# 1ª época: Calcula o erro para os 1ºs 10 registros e depois actualiza todos os pesos. Calcula o erro para os 2ºs 10 registros 
# e actualiza todos os pesos.
# 2ª época: Calcula o erro para os 1ºs 10 registros e depois actualiza todos os pesos da 1ª época. Calcula o erro para os 2ºs 
# 10 registros e actualiza todos os pesos da iteração anterior.

# Para um conjunto de dados de treino de 436 registros e batch_size de 6 registros:
# 426/6 = 71 iterações por época. 71 actualizações dos pesos por época

# Ajustar os dados de entrada de treino aos dados de saida de treino para treinar o modelo
model.fit(X_train, y_train, batch_size=1, epochs=2)

Train on 426 samples
Epoch 1/2
426/426 [==============================] - 1s 2ms/sample - loss: 5.7791 - binary_accuracy: 0.6174
Epoch 2/2
426/426 [==============================] - 1s 1ms/sample - loss: 5.6318 - binary_accuracy: 0.6315


In [13]:
# A avalição da binary_accuracy: 0.6385 está a ser feita no conjunto de treino, ou seja, está a fazer o treino e a avaliar no
# mesmo conjunto de dados. A avaliação deve ser feita para um conjunto de dados novos(não treinados).

### Previsão e Avaliação do Modelo ANN

In [14]:
# Prever os dados de teste para avaliar o modelo
predictions = model.predict(X_test)

In [15]:
# Passar os valores para verdadeiro ou falso
predictions = predictions > 0.5

In [16]:
data = pd.DataFrame(index = np.arange(0,143))
data["y_test"] = y_test.set_index(np.arange(0,143))
data["predictions"] = pd.DataFrame(predictions)
data.head()

,y_test,predictions
0,0,True
1,0,True
2,0,False
3,0,False
4,1,True


In [17]:
from sklearn.metrics import classification_report, confusion_matrix

In [18]:
#Relatório de classificação
#Ver como o modelo se ajusta para os dados de teste
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.84      0.81      0.83        53
           1       0.89      0.91      0.90        90

   micro avg       0.87      0.87      0.87       143
   macro avg       0.87      0.86      0.86       143
weighted avg       0.87      0.87      0.87       143



In [19]:
confusion = confusion_matrix(y_test,predictions)

In [20]:
#Matriz de confusão
outside_columns = ["Predicted",""]
outside_index = ["Actual",""]
inside = ["Benigno","Maligno"]

hier_columns = list(zip(outside_columns, inside))
hier_columns = pd.MultiIndex.from_tuples(hier_columns)

hier_index = list(zip(outside_index, inside))
hier_index = pd.MultiIndex.from_tuples(hier_index)

pd.DataFrame(confusion, columns=hier_columns, index = hier_index)

,,Predicted,
,,Benigno,Maligno
Actual,Benigno,43,10
,Maligno,8,82


In [21]:
# Com o TensorFlow 2.0
model.evaluate(X_test,y_test)

143/143 [==============================] - 0s 387us/sample - loss: 0.6338 - binary_accuracy: 0.8741


[0.6338372607096717, 0.8741259]

## Modelo Deep Neural Network (Modelo com mais de 1 camada oculta)

In [62]:
# Modelo com 2 camadas ocultas
# Na segunda camada oculta em geral coloca-se a mesma quantidade de neurónios do que na primeira, mas é necessário fazer testes
# para encontrar a melhor configuração.

# Ao se adicionar mais uma camada nem sempre leva a resultados melhores

model = models.Sequential()
model.add(layers.Dense(units=16, activation="relu", kernel_initializer="random_uniform", input_dim=30))
model.add(layers.Dense(units=16, activation="relu", kernel_initializer="random_uniform"))
model.add(layers.Dense(units=1, activation="sigmoid"))

### Compilação e Ajuste do modelo DNN

In [63]:

# lr: taxa de aprendizagem, serve para se obter o minimo global da função do erro. Com um learning rate grande, o ajustes dos 
# pesos é maior e obtem-se o minimo global mais rapido, mas se for demasiado alto pode acabar por se passar o valor do minimo 
# global e obter-se um valor maior.

# decay: quanto o learning rate é incrementado a cada actualização dos pesos. Por exmplo o lr começa em 0.01 e apos a 1ª 
# actualização dos pesos passa para 0.001. Permite colocar um lr mais alto porque dificilmente se encontra a melhor solução
# nas primeiras iterações. Assim não se corre tanto o risco de se passar o valor minimo global.

# clipvalue: Por defeito os valores dos pesos variam entre -0.5 e 0.5 para não sairem muito do padrão e não fiquem dispersos
# dentro do gradiente tentando encontrar o minimo global.

# caso se chega a valores de pesos menores e maiores do que -0.5 e 0.5(por defeito), esses pesos não são 

optimizer = keras.optimizers.Adam(lr=0.001, decay=0.0001, clipvalue=0.5)

In [24]:
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["binary_accuracy"])

In [25]:
model.fit(X_train, y_train, batch_size=10, epochs=100)

Train on 426 samples
Epoch 1/100
426/426 [==============================] - 0s 685us/sample - loss: 0.7935 - binary_accuracy: 0.6315
Epoch 2/100
426/426 [==============================] - 0s 190us/sample - loss: 0.5169 - binary_accuracy: 0.7230
Epoch 3/100
426/426 [==============================] - 0s 186us/sample - loss: 0.4789 - binary_accuracy: 0.7746
Epoch 4/100
426/426 [==============================] - 0s 193us/sample - loss: 0.5115 - binary_accuracy: 0.7629
Epoch 5/100
426/426 [==============================] - 0s 188us/sample - loss: 0.3988 - binary_accuracy: 0.8028
Epoch 6/100
426/426 [==============================] - 0s 177us/sample - loss: 0.3910 - binary_accuracy: 0.8286
Epoch 7/100
426/426 [==============================] - 0s 195us/sample - loss: 0.4248 - binary_accuracy: 0.8028
Epoch 8/100
426/426 [==============================] - 0s 193us/sample - loss: 0.3740 - binary_accuracy: 0.8638
Epoch 9/100
426/426 [==============================] - 0s 195us/sample - loss: 0.38

## Visialização dos Pesos da Rede

In [26]:
# Pesos da primeira camada oculta
# 30 entradas
# 16 neuronios na camada

pesos0 = model.layers[0].get_weights()
pesos1 = model.layers[1].get_weights()
pesos2 = model.layers[2].get_weights()

In [27]:
# O comprimento é 2 porque a primeira possição de tamanho de (30,16) tem os pesos da camada de entrada(30 entrada) a camada 
# oculta(16 neuronios).
# na segunda posição, como por defeito a rede neural é criada com bies, foi criado mais um neuronio na camada de entrada.
# Contem os valores dos pesos da ligação desse neuronio àcamada oculta(16 neurónios).

len(pesos0)

2

In [28]:
# O comprimento é 2 porque a primeira possição de tamanho de (16,16) tem os pesos da ligação da 1ª camada oculta(16 neuronios) 
# a 2ª camada oculta(16 neuronios).
# na segunda posição, como por defeito a rede neural é criada com bies, foi criado mais um neuronio na 1ª camada oculta.
# Contem os valores dos pesos da ligação desse neuronio à 2ª camada oculta(16 neurónios).

len(pesos1)

2

In [29]:
# O comprimento é 2 porque a primeira possição de tamanho de (16,1) tem os pesos da ligação da 2ª camada oculta(16 neuronios) 
# a camada de saida(1 neuronio).
# na segunda posição, como por defeito a rede neural é criada com bies, foi criado mais um neuronio na 2ª camada oculta.
# Contem os valores dos pesos da ligação desse neuronio à camada de saida(1 neuronio).

len(pesos2)

2

## Validação Cruzada (K-fold Cross Validation)

In [8]:
# Caso não se queira dividir o dataset num conjunto de dados de treino e de teste. Por exemplo em datasets com conjunto de dados
# dominante, corre-se o risco dessa separação de dados ser feita com o conjunto de treino ou teste a ter apenas dados do 
# tipo dominante. Ou quando se corre o risco de existirem dados importantes para o treino e que apenas ficam no conjunto de 
# de teste. Ou quando o conjunto de dados seja pequeno.

# Neste caso pode-se utilizar o k-fold cross validation.
# Ao utilizar-se o k fold cross validation, está-se a dividir o conjunto de dados em k sub conjuntos e a utilizar-se k-1 para 
# treino e o 1º conjunto para teste. Na segunda iteração o 2º sub conjunto é utilizado para teste e os restantes para treino.
# Este processo é realizado k vezes alternando de forma circular o subconjunto de teste.
# No fim calcula-se a media dos parâmetros(precision, accuracy) obtidos em cada iteração.
# Normalmente utiliza-se k=10.

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_validate

### Criar o modelo

In [65]:
### Criar o modelo de DNN numa função

def model():
    model = models.Sequential()
    model.add(layers.Dense(units=16, activation="relu", kernel_initializer="random_uniform", input_dim=30))
    model.add(layers.Dense(units=16, activation="relu", kernel_initializer="random_uniform"))
    model.add(layers.Dense(units=1, activation="sigmoid"))
    
    optimizer = keras.optimizers.Adam(lr=0.001, decay=0.0001, clipvalue=0.5)
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["binary_accuracy"])
    return model

In [66]:
model = KerasClassifier(build_fn=model, epochs=100, batch_size=10)

### Cross Validation

In [67]:
# cross validation

scoring = ['precision_weighted', 'recall_weighted', 'f1_weighted']

scores = cross_validate(estimator=model, X=X, y=y, cv=4, scoring="accuracy")

Train on 426 samples
Epoch 1/100
426/426 [==============================] - 0s 702us/sample - loss: 0.7379 - binary_accuracy: 0.5704
Epoch 2/100
426/426 [==============================] - 0s 206us/sample - loss: 0.5749 - binary_accuracy: 0.7465
Epoch 3/100
426/426 [==============================] - 0s 179us/sample - loss: 0.6862 - binary_accuracy: 0.7371
Epoch 4/100
426/426 [==============================] - 0s 195us/sample - loss: 0.5908 - binary_accuracy: 0.7676
Epoch 5/100
426/426 [==============================] - 0s 197us/sample - loss: 0.8082 - binary_accuracy: 0.7207
Epoch 6/100
426/426 [==============================] - 0s 193us/sample - loss: 0.4216 - binary_accuracy: 0.8451
Epoch 7/100
426/426 [==============================] - 0s 194us/sample - loss: 0.4830 - binary_accuracy: 0.8075
Epoch 8/100
426/426 [==============================] - 0s 226us/sample - loss: 0.4810 - binary_accuracy: 0.8451
Epoch 9/100
426/426 [==============================] - 0s 202us/sample - loss: 0.36

427/427 [==============================] - 0s 171us/sample - loss: 0.9483 - binary_accuracy: 0.8033
Epoch 46/100
427/427 [==============================] - 0s 192us/sample - loss: 0.8080 - binary_accuracy: 0.8431
Epoch 47/100
427/427 [==============================] - 0s 176us/sample - loss: 0.7953 - binary_accuracy: 0.8571
Epoch 48/100
427/427 [==============================] - 0s 163us/sample - loss: 1.0498 - binary_accuracy: 0.8080
Epoch 49/100
427/427 [==============================] - 0s 171us/sample - loss: 0.7960 - binary_accuracy: 0.8407
Epoch 50/100
427/427 [==============================] - 0s 169us/sample - loss: 0.8741 - binary_accuracy: 0.8689
Epoch 51/100
427/427 [==============================] - 0s 172us/sample - loss: 0.9883 - binary_accuracy: 0.8361
Epoch 52/100
427/427 [==============================] - 0s 169us/sample - loss: 1.0445 - binary_accuracy: 0.8314
Epoch 53/100
427/427 [==============================] - 0s 166us/sample - loss: 1.0215 - binary_accuracy: 0.8

427/427 [==============================] - 0s 176us/sample - loss: 0.8884 - binary_accuracy: 0.8595
Epoch 90/100
427/427 [==============================] - 0s 190us/sample - loss: 1.0950 - binary_accuracy: 0.8267
Epoch 91/100
427/427 [==============================] - 0s 176us/sample - loss: 1.0498 - binary_accuracy: 0.8407
Epoch 92/100
427/427 [==============================] - 0s 183us/sample - loss: 0.8655 - binary_accuracy: 0.8712
Epoch 93/100
427/427 [==============================] - 0s 174us/sample - loss: 0.9328 - binary_accuracy: 0.8407
Epoch 94/100
427/427 [==============================] - 0s 173us/sample - loss: 0.9447 - binary_accuracy: 0.8571
Epoch 95/100
427/427 [==============================] - 0s 170us/sample - loss: 0.8282 - binary_accuracy: 0.8876
Epoch 96/100
427/427 [==============================] - 0s 170us/sample - loss: 0.9162 - binary_accuracy: 0.8407
Epoch 97/100
427/427 [==============================] - 0s 172us/sample - loss: 1.1193 - binary_accuracy: 0.8

427/427 [==============================] - 0s 176us/sample - loss: 0.5713 - binary_accuracy: 0.8454
Epoch 62/100
427/427 [==============================] - 0s 197us/sample - loss: 0.5545 - binary_accuracy: 0.8478
Epoch 63/100
427/427 [==============================] - 0s 166us/sample - loss: 0.6777 - binary_accuracy: 0.8478
Epoch 64/100
427/427 [==============================] - 0s 161us/sample - loss: 0.5566 - binary_accuracy: 0.8548
Epoch 65/100
427/427 [==============================] - 0s 161us/sample - loss: 0.5566 - binary_accuracy: 0.8571
Epoch 66/100
427/427 [==============================] - 0s 168us/sample - loss: 0.5025 - binary_accuracy: 0.8595
Epoch 67/100
427/427 [==============================] - 0s 166us/sample - loss: 0.4687 - binary_accuracy: 0.8782
Epoch 68/100
427/427 [==============================] - 0s 166us/sample - loss: 0.4687 - binary_accuracy: 0.8665
Epoch 69/100
427/427 [==============================] - 0s 176us/sample - loss: 0.4518 - binary_accuracy: 0.8

In [68]:
# Valores obtidos para a accuracy atraves de cross validation
scores

{'fit_time': array([9.33485079, 9.43771601, 9.70452857, 8.64002442]),
 'score_time': array([0.05804253, 0.07506919, 0.05303812, 0.05113888]),
 'test_score': array([0.70629371, 0.80985915, 0.83098592, 0.75352113]),
 'train_score': array([0.85211268, 0.82669789, 0.88758782, 0.8501171 ])}

In [69]:
# accuracy média dos sub conjuntos de teste
np.mean(scores["test_score"])

0.7751649758692013

In [70]:
# desvio padrão obtido atraves dos valores da accuracy dos sub conjuntos de teste
# quanto maior este valor maior é a probabilidade de existir overfitting (o modelo não generaliza bem e adapta-se demasiado bem
# aos dados de treino, e quando se utilizam dados novos o modelo não se adapta bem)

np.std(scores["test_score"])

0.04881437118905278

## Overfitting(Dropout)

In [71]:
# underfitting - Ocorre da utilização de um algoritmo demasiado simplificado para o problema em analise. Obtem-se resultados
# maus no conjunto de dados de treino.
# overfitting - ocorre quando se utiliza um algoritmo demasiado complexo para o problema em analise. Obtem-se resultados bons
# no conjunto de treino e maus resultados no conjunto de dados de teste (o modelo adaptou-se bem demais/decorou o conjunto de
# dados de treino e não generaliza bem para conjuntos de dados novos).

# dropout - Ajuda a prevenir o overfitting(que em redes neurais tem grande probabilidade de acontecer devido aos muitos
# parametros que se utilizam).
# Coloca-se zeros em alguns valores/neuronios da camada de entrada de forma aleatoria. Escolhe-se a quantidade/percentagem de 
# neuronios que terão valor zero. Ao se colocar o valor zero estes neuronios não teram influencia no modelo. Normalmente 
# utiliza-se uma percentagem entre 20% a 30%. Se for demasiado elevado(maior que 50%) o modelo pode entrar em underfitting, 
# porque não consegue aprender, pois tem demasiadas entradas que não são consideradas.
# Como se utilizam menos variaveis é mais dificil haver alguma que não tenham sido treinada e que o modelo não consiga prever.

# https://stats.stackexchange.com/questions/299292/dropout-makes-performance-worse

### Criar o modelo com as camadas Dropout

In [72]:
### Criar o modelo de DNN numa função e com o dropout

# 1 - 20% dos neuronios da camada de entrada terão valor zero
# 2 - 20% dos neuronios da 1ª camada oculta terão valor zero

def model():
    model = models.Sequential()
    model.add(layers.Dense(units=16, activation="relu", kernel_initializer="random_uniform", input_dim=30))
    model.add(layers.Dropout(0.2)) 
    model.add(layers.Dense(units=16, activation="relu", kernel_initializer="random_uniform"))
    model.add(layers.Dropout(0.2)) 
    model.add(layers.Dense(units=1, activation="sigmoid"))
    
    optimizer = keras.optimizers.Adam(lr=0.001, decay=0.0001, clipvalue=0.5)
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["binary_accuracy"])
    return model

In [73]:
model = KerasClassifier(build_fn=model, epochs=100, batch_size=10)

### Cross Validation

In [74]:
# cross validation

scoring = ['precision_weighted', 'recall_weighted', 'f1_weighted']

scores = cross_validate(estimator=model, X=X, y=y, cv=4, scoring="accuracy")

Train on 426 samples
Epoch 1/100
426/426 [==============================] - 0s 1ms/sample - loss: 1.1774 - binary_accuracy: 0.6056
Epoch 2/100
426/426 [==============================] - 0s 230us/sample - loss: 0.7169 - binary_accuracy: 0.6948
Epoch 3/100
426/426 [==============================] - 0s 206us/sample - loss: 0.6070 - binary_accuracy: 0.7089
Epoch 4/100
426/426 [==============================] - 0s 216us/sample - loss: 0.6236 - binary_accuracy: 0.7371
Epoch 5/100
426/426 [==============================] - 0s 265us/sample - loss: 0.6299 - binary_accuracy: 0.7136
Epoch 6/100
426/426 [==============================] - 0s 237us/sample - loss: 0.6769 - binary_accuracy: 0.6784
Epoch 7/100
426/426 [==============================] - 0s 207us/sample - loss: 0.6429 - binary_accuracy: 0.7300
Epoch 8/100
426/426 [==============================] - 0s 215us/sample - loss: 0.6582 - binary_accuracy: 0.7418
Epoch 9/100
426/426 [==============================] - 0s 223us/sample - loss: 0.7149

427/427 [==============================] - 0s 251us/sample - loss: 0.4235 - binary_accuracy: 0.8478
Epoch 46/100
427/427 [==============================] - 0s 195us/sample - loss: 0.4198 - binary_accuracy: 0.8244
Epoch 47/100
427/427 [==============================] - 0s 190us/sample - loss: 0.4348 - binary_accuracy: 0.8548
Epoch 48/100
427/427 [==============================] - 0s 215us/sample - loss: 0.5099 - binary_accuracy: 0.8197
Epoch 49/100
427/427 [==============================] - 0s 182us/sample - loss: 0.4954 - binary_accuracy: 0.8126
Epoch 50/100
427/427 [==============================] - 0s 190us/sample - loss: 0.4951 - binary_accuracy: 0.8080
Epoch 51/100
427/427 [==============================] - 0s 194us/sample - loss: 0.4173 - binary_accuracy: 0.8571
Epoch 52/100
427/427 [==============================] - 0s 194us/sample - loss: 0.4627 - binary_accuracy: 0.8314
Epoch 53/100
427/427 [==============================] - 0s 190us/sample - loss: 0.4940 - binary_accuracy: 0.8

427/427 [==============================] - 0s 209us/sample - loss: 0.6445 - binary_accuracy: 0.8665
Epoch 90/100
427/427 [==============================] - 0s 230us/sample - loss: 0.7878 - binary_accuracy: 0.8501
Epoch 91/100
427/427 [==============================] - 0s 198us/sample - loss: 0.6293 - binary_accuracy: 0.8829
Epoch 92/100
427/427 [==============================] - 0s 209us/sample - loss: 0.6472 - binary_accuracy: 0.8970
Epoch 93/100
427/427 [==============================] - 0s 213us/sample - loss: 0.6965 - binary_accuracy: 0.8431
Epoch 94/100
427/427 [==============================] - 0s 216us/sample - loss: 0.6964 - binary_accuracy: 0.8759
Epoch 95/100
427/427 [==============================] - 0s 284us/sample - loss: 0.6592 - binary_accuracy: 0.8618
Epoch 96/100
427/427 [==============================] - 0s 307us/sample - loss: 0.6226 - binary_accuracy: 0.8665
Epoch 97/100
427/427 [==============================] - 0s 237us/sample - loss: 0.7811 - binary_accuracy: 0.8

In [75]:
# Valores obtidos para a accuracy atraves de cross validation
scores

{'fit_time': array([11.6330204 , 11.03969073, 10.57655263, 11.17863917]),
 'score_time': array([0.09374332, 0.09057546, 0.08735466, 0.10421395]),
 'test_score': array([0.67132867, 0.80985915, 0.88028169, 0.84507042]),
 'train_score': array([0.83333333, 0.91334895, 0.89929742, 0.91334895])}

In [76]:
# accuracy média dos sub conjuntos de teste
np.mean(scores["test_score"])

0.8016349847335762

In [77]:
# desvio padrão obtido atraves dos valores da accuracy dos sub conjuntos de teste
# o desvio padrão diminui com o uso do dropout, existe menos variabilidade nos resultados obtidos para o conjunto de teste
np.std(scores["test_score"])

0.07924536866124658

## Ajuste de parâmetros

In [78]:
from sklearn.model_selection import GridSearchCV

### Criar o modelo com as camadas Dropout e com vários valores para os parametros

In [85]:
### Criar o modelo de DNN numa função, com dropout e ajuste de parâmetros

def model(optimizer, loss, kernel_initializer, activation, neurons):
    model = models.Sequential()
    model.add(layers.Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer, input_dim=30))
    model.add(layers.Dropout(0.2)) 
    model.add(layers.Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer))
    model.add(layers.Dropout(0.2)) 
    model.add(layers.Dense(units=1, activation="sigmoid"))
    
    model.compile(optimizer=optimizer, loss=loss, metrics=["binary_accuracy"])
    return model

In [86]:
model = KerasClassifier(build_fn=model )

In [87]:
parametros = {'batch_size': [10, 30],
              'epochs': [50, 100],
              'optimizer': ['adam', 'sgd'],
              'loss': ['binary_crossentropy', 'hinge'],
              'kernel_initializer': ['random_uniform', 'normal'],
              'activation': ['relu', 'tanh'],
              'neurons': [16, 8]}

In [88]:
#  Construção da grelha
grid = GridSearchCV(estimator=model, param_grid=parametros, scoring="accuracy", cv=4, refit=True)

In [ ]:
# Ajustar o modelo com os parametros escolhidos e para o conjunto de dados total
# Demora cerca de 8 horas a encontrar os melhores parametros
grid.fit(X,y)

Train on 426 samples
Epoch 1/50
426/426 [==============================] - 0s 915us/sample - loss: 1.2350 - binary_accuracy: 0.5939
Epoch 2/50
426/426 [==============================] - 0s 228us/sample - loss: 0.6273 - binary_accuracy: 0.6761
Epoch 3/50
426/426 [==============================] - 0s 189us/sample - loss: 0.5472 - binary_accuracy: 0.6549
Epoch 4/50
426/426 [==============================] - 0s 226us/sample - loss: 0.5231 - binary_accuracy: 0.7300
Epoch 5/50
426/426 [==============================] - 0s 233us/sample - loss: 0.4947 - binary_accuracy: 0.7300
Epoch 6/50
426/426 [==============================] - 0s 226us/sample - loss: 0.5085 - binary_accuracy: 0.7136
Epoch 7/50
426/426 [==============================] - 0s 211us/sample - loss: 0.4788 - binary_accuracy: 0.7629
Epoch 8/50
426/426 [==============================] - 0s 221us/sample - loss: 0.4416 - binary_accuracy: 0.7981
Epoch 9/50
426/426 [==============================] - 0s 223us/sample - loss: 0.4239 - bina

Epoch 24/50
427/427 [==============================] - 0s 204us/sample - loss: 0.2740 - binary_accuracy: 0.8946
Epoch 25/50
427/427 [==============================] - 0s 234us/sample - loss: 0.3480 - binary_accuracy: 0.8618
Epoch 26/50
427/427 [==============================] - 0s 197us/sample - loss: 0.2970 - binary_accuracy: 0.9040
Epoch 27/50
427/427 [==============================] - 0s 206us/sample - loss: 0.2876 - binary_accuracy: 0.8899
Epoch 28/50
427/427 [==============================] - 0s 192us/sample - loss: 0.2647 - binary_accuracy: 0.9063
Epoch 29/50
427/427 [==============================] - 0s 192us/sample - loss: 0.2842 - binary_accuracy: 0.9016
Epoch 30/50
427/427 [==============================] - 0s 192us/sample - loss: 0.2627 - binary_accuracy: 0.9016
Epoch 31/50
427/427 [==============================] - 0s 190us/sample - loss: 0.3050 - binary_accuracy: 0.8829
Epoch 32/50
427/427 [==============================] - 0s 188us/sample - loss: 0.2715 - binary_accuracy:

427/427 [==============================] - 0s 206us/sample - loss: 0.2647 - binary_accuracy: 0.8970
Epoch 48/50
427/427 [==============================] - 0s 211us/sample - loss: 0.2603 - binary_accuracy: 0.8852
Epoch 49/50
427/427 [==============================] - 0s 194us/sample - loss: 0.2427 - binary_accuracy: 0.9040
Epoch 50/50
427/427 [==============================] - 0s 196us/sample - loss: 0.2272 - binary_accuracy: 0.9087
Train on 427 samples
Epoch 1/50
427/427 [==============================] - 1s 1ms/sample - loss: 1.5571 - binary_accuracy: 0.5597
Epoch 2/50
427/427 [==============================] - 0s 161us/sample - loss: 0.6417 - binary_accuracy: 0.6206
Epoch 3/50
427/427 [==============================] - 0s 148us/sample - loss: 0.5420 - binary_accuracy: 0.6440
Epoch 4/50
427/427 [==============================] - 0s 171us/sample - loss: 0.5407 - binary_accuracy: 0.6721
Epoch 5/50
427/427 [==============================] - 0s 158us/sample - loss: 0.5445 - binary_accurac

426/426 [==============================] - 0s 195us/sample - loss: 4.5722 - binary_accuracy: 0.6925
Epoch 21/50
426/426 [==============================] - 0s 237us/sample - loss: 4.3507 - binary_accuracy: 0.6925
Epoch 22/50
426/426 [==============================] - 0s 216us/sample - loss: 4.4832 - binary_accuracy: 0.6925
Epoch 23/50
426/426 [==============================] - 0s 174us/sample - loss: 4.6306 - binary_accuracy: 0.6948
Epoch 24/50
426/426 [==============================] - 0s 230us/sample - loss: 4.6668 - binary_accuracy: 0.6901
Epoch 25/50
426/426 [==============================] - 0s 282us/sample - loss: 4.6048 - binary_accuracy: 0.6948
Epoch 26/50
426/426 [==============================] - 0s 204us/sample - loss: 4.2264 - binary_accuracy: 0.6948
Epoch 27/50
426/426 [==============================] - 0s 176us/sample - loss: 3.8508 - binary_accuracy: 0.6948
Epoch 28/50
426/426 [==============================] - 0s 180us/sample - loss: 3.7604 - binary_accuracy: 0.6948
Epoc

427/427 [==============================] - 0s 171us/sample - loss: 6.6905 - binary_accuracy: 0.4707
Epoch 15/50
427/427 [==============================] - 0s 218us/sample - loss: 7.3966 - binary_accuracy: 0.4215
Epoch 16/50
427/427 [==============================] - 0s 146us/sample - loss: 7.2624 - binary_accuracy: 0.4778
Epoch 17/50
427/427 [==============================] - 0s 137us/sample - loss: 6.9801 - binary_accuracy: 0.4684
Epoch 18/50
427/427 [==============================] - 0s 151us/sample - loss: 7.0352 - binary_accuracy: 0.4590
Epoch 19/50
427/427 [==============================] - 0s 132us/sample - loss: 7.1854 - binary_accuracy: 0.4707
Epoch 20/50
427/427 [==============================] - 0s 131us/sample - loss: 7.4318 - binary_accuracy: 0.4286
Epoch 21/50
427/427 [==============================] - 0s 137us/sample - loss: 7.4950 - binary_accuracy: 0.4426
Epoch 22/50
427/427 [==============================] - 0s 146us/sample - loss: 7.6751 - binary_accuracy: 0.4614
Epoc

...

427/427 [==============================] - 0s 156us/sample - loss: 0.8302 - binary_accuracy: 0.5855
Epoch 13/50
427/427 [==============================] - 0s 155us/sample - loss: 0.8290 - binary_accuracy: 0.5855
Epoch 14/50
427/427 [==============================] - 0s 156us/sample - loss: 0.8291 - binary_accuracy: 0.5855
Epoch 15/50
427/427 [==============================] - 0s 152us/sample - loss: 0.8291 - binary_accuracy: 0.5855
Epoch 16/50
427/427 [==============================] - 0s 167us/sample - loss: 0.8291 - binary_accuracy: 0.5855
Epoch 17/50
427/427 [==============================] - 0s 151us/sample - loss: 0.8290 - binary_accuracy: 0.5855
Epoch 18/50
427/427 [==============================] - 0s 157us/sample - loss: 0.8292 - binary_accuracy: 0.5855
Epoch 19/50
427/427 [==============================] - 0s 155us/sample - loss: 0.8291 - binary_accuracy: 0.5855
Epoch 20/50
427/427 [==============================] - 0s 157us/sample - loss: 0.8295 - binary_accuracy: 0.5855
Epoc

427/427 [==============================] - 0s 253us/sample - loss: 0.7836 - binary_accuracy: 0.6464
Epoch 9/50
427/427 [==============================] - 0s 260us/sample - loss: 0.8008 - binary_accuracy: 0.6136
Epoch 10/50
427/427 [==============================] - 0s 225us/sample - loss: 0.7804 - binary_accuracy: 0.6487
Epoch 11/50
427/427 [==============================] - 0s 173us/sample - loss: 0.7978 - binary_accuracy: 0.6206
Epoch 12/50
427/427 [==============================] - 0s 108us/sample - loss: 0.8053 - binary_accuracy: 0.6042
Epoch 13/50
427/427 [==============================] - 0s 110us/sample - loss: 0.8099 - binary_accuracy: 0.6066
Epoch 14/50
427/427 [==============================] - 0s 165us/sample - loss: 0.8152 - binary_accuracy: 0.6042
Epoch 15/50
427/427 [==============================] - 0s 166us/sample - loss: 0.7918 - binary_accuracy: 0.6230
Epoch 16/50
427/427 [==============================] - 0s 139us/sample - loss: 0.8171 - binary_accuracy: 0.5995
Epoch

427/427 [==============================] - 0s 110us/sample - loss: 0.7032 - binary_accuracy: 0.6534
Epoch 5/50
427/427 [==============================] - 0s 146us/sample - loss: 0.7117 - binary_accuracy: 0.6440
Epoch 6/50
427/427 [==============================] - 0s 110us/sample - loss: 0.7109 - binary_accuracy: 0.6487
Epoch 7/50
427/427 [==============================] - 0s 163us/sample - loss: 0.7021 - binary_accuracy: 0.6511
Epoch 8/50
427/427 [==============================] - 0s 209us/sample - loss: 0.7018 - binary_accuracy: 0.6464
Epoch 9/50
427/427 [==============================] - 0s 190us/sample - loss: 0.7039 - binary_accuracy: 0.6487
Epoch 10/50
427/427 [==============================] - 0s 152us/sample - loss: 0.7048 - binary_accuracy: 0.6511
Epoch 11/50
427/427 [==============================] - 0s 97us/sample - loss: 0.6979 - binary_accuracy: 0.6511
Epoch 12/50
427/427 [==============================] - 0s 146us/sample - loss: 0.7002 - binary_accuracy: 0.6511
Epoch 13/5

427/427 [==============================] - 0s 199us/sample - loss: 0.8407 - binary_accuracy: 0.5855
Train on 426 samples
Epoch 1/50
426/426 [==============================] - 5s 12ms/sample - loss: 2.9158 - binary_accuracy: 0.5587
Epoch 2/50
426/426 [==============================] - 0s 174us/sample - loss: 1.2150 - binary_accuracy: 0.5939
Epoch 3/50
426/426 [==============================] - 0s 178us/sample - loss: 0.7413 - binary_accuracy: 0.6761
Epoch 4/50
426/426 [==============================] - 0s 230us/sample - loss: 0.6804 - binary_accuracy: 0.6620
Epoch 5/50
426/426 [==============================] - 0s 204us/sample - loss: 0.6955 - binary_accuracy: 0.6549
Epoch 6/50
426/426 [==============================] - 0s 188us/sample - loss: 0.6013 - binary_accuracy: 0.6667
Epoch 7/50
426/426 [==============================] - 0s 188us/sample - loss: 0.6401 - binary_accuracy: 0.6831
Epoch 8/50
426/426 [==============================] - 0s 200us/sample - loss: 0.5899 - binary_accuracy:

427/427 [==============================] - 0s 185us/sample - loss: 0.2760 - binary_accuracy: 0.8876
Epoch 46/50
427/427 [==============================] - 0s 199us/sample - loss: 0.2831 - binary_accuracy: 0.8806
Epoch 47/50
427/427 [==============================] - 0s 180us/sample - loss: 0.2715 - binary_accuracy: 0.8970
Epoch 48/50
427/427 [==============================] - 0s 234us/sample - loss: 0.2349 - binary_accuracy: 0.9063
Epoch 49/50
427/427 [==============================] - 0s 187us/sample - loss: 0.2554 - binary_accuracy: 0.9063
Epoch 50/50
427/427 [==============================] - 0s 239us/sample - loss: 0.2754 - binary_accuracy: 0.8852


In [ ]:
# Os melhores parametros para o modelo
grid.best_params_

In [ ]:
# A melhor previsão é obtida com os seguintes hyperparâmetros
grid.best_estimator_

## Classificação de apenas 1 registro (Após o modelo estar pronto para produção)

### Criar o modelo final

In [238]:
# Modelo final
# 8 neurónios e kernel_initializer="normal" porque foram os parametros encontrados atraves da gridsearch que optimizam o modelo.

model = models.Sequential()
model.add(layers.Dense(units=8, activation="relu", kernel_initializer="normal", input_dim=30))
model.add(layers.Dropout(0.2)) 
model.add(layers.Dense(units=8, activation="relu", kernel_initializer="normal"))
model.add(layers.Dropout(0.2)) 
model.add(layers.Dense(units=1, activation="sigmoid"))     

### Compilação e Ajuste do modelo final

In [239]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["binary_accuracy"])

In [240]:
model.fit(X, y, epochs=100, batch_size=10)

Train on 569 samples
Epoch 1/100
569/569 [==============================] - 0s 734us/sample - loss: 2.4770 - binary_accuracy: 0.5905
Epoch 2/100
569/569 [==============================] - 0s 213us/sample - loss: 0.8328 - binary_accuracy: 0.5466
Epoch 3/100
569/569 [==============================] - 0s 202us/sample - loss: 0.6206 - binary_accuracy: 0.5923
Epoch 4/100
569/569 [==============================] - 0s 199us/sample - loss: 0.6545 - binary_accuracy: 0.6169
Epoch 5/100
569/569 [==============================] - 0s 216us/sample - loss: 0.6337 - binary_accuracy: 0.6239
Epoch 6/100
569/569 [==============================] - 0s 218us/sample - loss: 0.5685 - binary_accuracy: 0.6257
Epoch 7/100
569/569 [==============================] - 0s 202us/sample - loss: 0.6132 - binary_accuracy: 0.6221
Epoch 8/100
569/569 [==============================] - 0s 209us/sample - loss: 0.5931 - binary_accuracy: 0.6239
Epoch 9/100
569/569 [==============================] - 0s 213us/sample - loss: 0.59

### Previsão com o modelo final

In [241]:
# novo entrada de dados para prever
data = np.array([[15.80, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178,
                  0.20, 0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.015,
                  0.03, 0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185,
                  0.84, 158, 0.363]])

In [242]:
# previsão para os novos dados
prediction = model.predict(data)
prediction

array([[1.]], dtype=float32)

In [243]:
prediction = pred > 0.5
prediction

array([[ True]])

## Gravar a Rede Neural

In [244]:
# gravar a rede neural final(já treinada com os melhores parâmetros, pesos e estrutura) para não ser necessário fazer de novo 
# o treino. Assim já não é necessário fazer o treino quando se quer prever um novo registro.

### Criar o modelo final

In [245]:
# Modelo final
# 8 neurónios e kernel_initializer="normal" porque foram os parametros encontrados atraves da gridsearch que optimizam o modelo.

model = models.Sequential()
model.add(layers.Dense(units=8, activation="relu", kernel_initializer="normal", input_dim=30))
model.add(layers.Dropout(0.2)) 
model.add(layers.Dense(units=8, activation="relu", kernel_initializer="normal"))
model.add(layers.Dropout(0.2)) 
model.add(layers.Dense(units=1, activation="sigmoid"))  

### Compilação e Ajuste do modelo final

In [246]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["binary_accuracy"])

In [247]:
model.fit(X, y, epochs=100, batch_size=10)

Train on 569 samples
Epoch 1/100
569/569 [==============================] - 0s 685us/sample - loss: 3.3517 - binary_accuracy: 0.4464
Epoch 2/100
569/569 [==============================] - 0s 227us/sample - loss: 1.1978 - binary_accuracy: 0.5097
Epoch 3/100
569/569 [==============================] - 0s 215us/sample - loss: 0.7641 - binary_accuracy: 0.5571
Epoch 4/100
569/569 [==============================] - 0s 200us/sample - loss: 0.6870 - binary_accuracy: 0.6011
Epoch 5/100
569/569 [==============================] - 0s 196us/sample - loss: 0.6494 - binary_accuracy: 0.5888
Epoch 6/100
569/569 [==============================] - 0s 209us/sample - loss: 0.6068 - binary_accuracy: 0.6415
Epoch 7/100
569/569 [==============================] - 0s 199us/sample - loss: 0.6236 - binary_accuracy: 0.6591
Epoch 8/100
569/569 [==============================] - 0s 218us/sample - loss: 0.5913 - binary_accuracy: 0.6714
Epoch 9/100
569/569 [==============================] - 0s 216us/sample - loss: 0.54

### Gravar o modelo final

In [248]:
model_json = model.to_json()

In [249]:
# gravar o modelo no formato json
with open('model_breast.json', 'w') as json_file:
    json_file.write(model_json)

In [250]:
# Se o h5 não estiver instalado: pip install h5py
# gravar os pesos no formato h5
model.save_weights('weights_breast.h5')

In [251]:
# gravar o modelo no formato h5
model.save('model_breast.h5')

## Carregar o modelo guardado

In [252]:
from tensorflow.keras.models import model_from_json

In [253]:
# carregar o modelo guardado no formato json
arquivo = open('model_breast.json', 'r')
estrutura_rede = arquivo.read()
arquivo.close()

saveModel = model_from_json(estrutura_rede)

In [254]:
# carregar o modelo guardado no formato h5
#saveModel = models.load_model("model_breast.h5")

In [255]:
# carregar os pesos do modelo guardado
saveModel.load_weights("weights_breast.h5")

### Previsão com o modelo final guardado

In [256]:
# nova entrada de dados para prever
data = np.array([[15.80, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178,
                  0.20, 0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.015,
                  0.03, 0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185,
                  0.84, 158, 0.363]])

In [257]:
# previsão para o novo registro de dados
saveModel.predict(data)

array([[0.9995981]], dtype=float32)

In [258]:
prediction = pred > 0.5
prediction

array([[ True]])

### Avaliação do modelo guardado

In [259]:
saveModel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

In [260]:
saveModel.evaluate(X, y)

569/569 [==============================] - 0s 188us/sample - loss: 0.1771 - binary_accuracy: 0.9367


[0.17708493674890857, 0.9367311]